## Purpose: 
Demonstration of how an RL policy could learn intraday heuristics. 
This is a toy Q-learning demo with discrete actions {charge, idle, discharge} and a simple reward. 
I have used this to explain the RL pipeline — environment, reward shaping, domain randomization, shadow-trading & safety.